In [ ]:
from pylab import *
import os
import sys
from sklearn.mixture import GaussianMixture
import pickle
from PIL import Image

In [ ]:
dataset_dir = 'rgbd_dataset_freiburg3_long_office_household'
depth_dir = 'depth'
gmm_dir = 'gmm_qqvga'

# og 
fx = 525.0  # focal length x
fy = 525.0  # focal length y
cx = 319.5  # optical center x
cy = 239.5  # optical center y

# fri3 
fx = 535.4  # focal length x
fy = 539.2  # focal length y
cx = 320.1  # optical center x
cy = 247.6  # optical center y
factor = 5000 # for the 16-bit PNG files
# OR: factor = 1 # for the 32-bit float images in the ROS bag files

full_depth_dir = os.path.join(dataset_dir,depth_dir)
full_gmm_dir = os.path.join(dataset_dir,gmm_dir)
if not os.path.exists(full_gmm_dir):
    os.mkdir(full_gmm_dir)

In [ ]:
y_grid = np.repeat(np.arange(480)[:,None],640,1)
x_grid = np.repeat(np.arange(640)[None,:],480,0)

def img_to_pts(img):
    Z = img/factor
    X = (x_grid-cx) * Z /fx
    Y = (y_grid-cy) * Z /fy
    #for v in range(depth_image.height):
    #  for u in range(depth_image.width):
    #    Z = depth_image[v,u] / factor;
    #    X = (u - cx) * Z / fx;
    #    Y = (v - cy) * Z / fy;
    X = X[::4,::4]
    Y = Y[::4,::4]
    Z = Z[::4,::4]
    print(X.shape)

    xl = X[Z > 0]
    yl = Y[Z > 0]
    zl = Z[Z > 0]
    n = zl.shape[0]
    print(n)
    idx = np.random.randint(n, size=n//16)

    return np.vstack([xl,yl,zl]).T

def load_data(file):
    image = Image.open(file)   
    pixel = np.array(image)
    return img_to_pts(pixel)

for fl in sorted(os.listdir(full_depth_dir)):
    name,ext = os.path.splitext(fl)
    new_name = name + '.pkl'
    
    clf = GaussianMixture(100)
    data = load_data(os.path.join(full_depth_dir,fl))
    t1 = time.time()
    clf.fit(data)
    print(time.time()-t1)
    with open(os.path.join(full_gmm_dir,new_name),'wb') as fp:
        pickle.dump(clf,fp)
    

In [ ]:
name,ext